<a href="https://colab.research.google.com/github/denis5ds/IA-sis420/blob/main/Primerparcial/primerparcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset asignado D1(34)

conectandi al drive

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Librerias que se usan

In [102]:
#librerias que se van a utilizar
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import pandas as pd # para el preprocesamiento
from sklearn import preprocessing
import csv
%matplotlib inline


cargamos los datos del dataset
https://www.kaggle.com/datasets/mikolajbabula/adult-census-income-data

1   Inserte atributos que contengan, caracteres, texto, valores booleanos relacionados a la temática del dataset elegido.

crea las 3 columnas sisteticas y tambien realiza el mapeo de llas




In [103]:
# Cargado de datos
df = pd.read_csv("/content/drive/MyDrive/IA/datasets/adult.csv", sep=",")

# Mapeo 1
mapeo1 = {"Private": 1, "Self-emp-not-inc": 2, "Local-gov":3, "?":4, "State-gov":5, "Self-emp-inc":6, "Federal-gov":7, "Without-pay":8, "Never-worked":9}
df["workclass"] = df["workclass"].replace(mapeo1)

# Mapeo 2
mapeo2 = {"HS-grad": 1, "Some-college": 2, "Bachelors":3, "Masters":4, "Assoc-voc":5, "11th":6, "Assoc-acdm":7, "10th":8, "7th-8th":9, "Prof-school":10,
          "9th":11, "12th":12, "Doctorate":13, "5th-6th":14, "1st-4th":15, "Preschool":16}
df["education"] = df["education"].replace(mapeo2)

# Mapeo 3
mapeo3 = {"Married-civ-spouse": 1, "Never-married": 2, "Divorced":3, "Separated":4, "Widowed":5, "Married-spouse-absent":6, "Married-AF-spouse":7}
df["marital.status"] = df["marital.status"].replace(mapeo3)

# Mapeo 4
mapeo4 = {"Prof-specialty": 1, "Craft-repair": 2, "Exec-managerial":3, "Adm-clerical":4, "Sales":5, "Other-service":6, "Machine-op-inspct":7, "?":8,
          "Transport-moving":9, "Handlers-cleaners":10, "Farming-fishing":11, "Tech-support":12, "Protective-serv":13, "Priv-house-serv":14, "Armed-Forces":15}
df["occupation"] = df["occupation"].replace(mapeo4)

# Mapeo 5
mapeo5 = {"Husband": 1, "Not-in-family": 2, "Own-child":3, "Unmarried":4, "Wife":5, "Other-relative":6}
df["relationship"] = df["relationship"].replace(mapeo5)

# Mapeo 6
mapeo6 = {"White": 1, "Black": 2, "Asian-Pac-Islander":3, "Amer-Indian-Eskimo":4, "Other":5}
df["race"] = df["race"].replace(mapeo6)

# Mapeo 7
mapeo7 = {"Male": 1, "Female": 2}
df["sex"] = df["sex"].replace(mapeo7)

# Mapeo 8
mapeo8 = {"?": 0,"United-States": 0,"Mexico": 0,"Philippines": 0,"Germany": 0,"Canada": 0,"Puerto-Rico": 0,"El-Salvador": 0,"India": 0,"Cuba": 0,"England": 0,"Jamaica": 0,
          "South": 0,"China": 0, "Italy": 1, "Dominican-Republic": 1, "Vietnam": 1, "Guatemala": 1, "Japan": 1, "Poland": 1, "Columbia": 1, "Taiwan": 1, "Haiti": 1,
          "Iran": 1, "Portugal": 1, "Nicaragua": 1, "Peru": 1, "Greece": 1, "Holand-Netherlands": 2, "France": 2, "Ecuador": 2, "Ireland": 2, "Hong": 2,
          "Trinadad&Tobago": 2, "Cambodia": 2, "Laos": 2, "Thailand": 2, "Yugoslavia": 2, "Outlying-US(Guam-USVI-etc)": 2, "Honduras": 2, "Hungary": 2, "Scotland": 2}
df["native.country"] = df["native.country"].replace(mapeo8)

# Se eliminan los datos que entran en conflicto con las operaciones
df = df.drop('income', axis=1)
df = df.drop('fnlwgt', axis=1)
df.drop_duplicates()
df.dropna(axis=0, thresh=2)  # Elimina filas con 2 o más valores nulos.


###################################################################################################
#Agregando las nuevas 3 columnas sinteticas

# Columna de texto (grupo de edad según la columna "age")
def clasificar_edad(age):
    if age < 25:
        return 'Joven'
    elif 25 <= age < 50:
        return 'Adulto'
    else:
        return 'Mayor'

df['age_group'] = df['age'].apply(clasificar_edad)
#mapeo9
mapeo9 = {"Joven":1,"Adulto":2,"Mayor":3}
df["age_group"] = df["age_group"].replace(mapeo9)


# Columna booleana (si trabaja más de 40 horas por semana)
df['high_hours_per_week'] = df['hours.per.week'] > 40

# Columna de opciones (si la persona fuma o bebe)
df['smokes_or_drinks'] = np.random.choice(['Sí', 'No', 'No se sabe'], size=len(df))

# Reemplazar los valores de 'Sí', 'No', 'No se sabe' por números
df['smokes_or_drinks'] = df['smokes_or_drinks'].replace({'Sí': 2, 'No': 1, 'No se sabe': 0})

# Reemplazar los valores True por 2 y False por 1 en la columna high_hours_per_week
df['high_hours_per_week'] = df['high_hours_per_week'].replace({True: 2, False: 1})

# Verificar que las columnas se hayan agregado correctamente
####################################################################################################
print(df.head())

   age  workclass  education  education.num  marital.status  occupation  \
0   90          4          1              9               5           8   
1   82          1          1              9               5           3   
2   66          4          2             10               5           8   
3   54          1          9              4               3           7   
4   41          1          2             10               4           1   

   relationship  race  sex  capital.gain  capital.loss  hours.per.week  \
0             2     1    2             0          4356              40   
1             2     1    2             0          4356              18   
2             4     2    2             0          4356              40   
3             4     1    2             0          3900              40   
4             3     1    2             0          3900              40   

   native.country  age_group  high_hours_per_week  smokes_or_drinks  
0               0          3      

ver cantidad de datos de cada clase

In [104]:
df["native.country"].value_counts()

,count
native.country,
0,31593
1,724
2,244


informacion del dataset modificado

In [105]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  32561 non-null  int64
 1   workclass            32561 non-null  int64
 2   education            32561 non-null  int64
 3   education.num        32561 non-null  int64
 4   marital.status       32561 non-null  int64
 5   occupation           32561 non-null  int64
 6   relationship         32561 non-null  int64
 7   race                 32561 non-null  int64
 8   sex                  32561 non-null  int64
 9   capital.gain         32561 non-null  int64
 10  capital.loss         32561 non-null  int64
 11  hours.per.week       32561 non-null  int64
 12  native.country       32561 non-null  int64
 13  age_group            32561 non-null  int64
 14  high_hours_per_week  32561 non-null  int64
 15  smokes_or_drinks     32561 non-null  int64
dtypes: int64(16)
memory us

duplicacion de los datos

In [106]:
# 1. Obtener estadísticas descriptivas del DataFrame original
mean_values = df.mean()
std_values = df.std()

# 2. Crear una función para generar datos sintéticos
def generar_datos_sinteticos(df, num_nuevos):
    datos_nuevos = []
    for _ in range(num_nuevos):
        nuevo_dato = {}
        for col in df.columns:
            if df[col].dtype == 'object':  # Para columnas categóricas
                nuevo_dato[col] = np.random.choice(df[col].unique())
            elif df[col].dtype == 'int64' or df[col].dtype == 'float64':  # Para columnas numéricas
                mean = mean_values[col]
                std = std_values[col]
                nuevo_dato[col] = np.random.normal(mean, std)
                # Ajustar para mantener la coherencia
                if col in ['hours.per.week', 'capital.gain', 'capital.loss']:
                    nuevo_dato[col] = max(0, nuevo_dato[col])  # No valores negativos
            else:
                nuevo_dato[col] = np.nan
        datos_nuevos.append(nuevo_dato)
    return pd.DataFrame(datos_nuevos)

# Generar datos sintéticos
num_nuevos = len(df)  # Duplicar el número de datos
df_nuevos = generar_datos_sinteticos(df, num_nuevos)

# Combinar datos originales con los nuevos
df_completo = pd.concat([df, df_nuevos], ignore_index=True)

# Verificar el resultado
print(df_completo.shape)

(65122, 16)


graficos de distrivucion del dataset antiguo y el dataset modificado y duplicado con datos sintetico

In [107]:
df.dropna(axis=0, thresh=2)

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,age_group,high_hours_per_week,smokes_or_drinks
0,90,4,1,9,5,8,2,1,2,0,4356,40,0,3,1,0
1,82,1,1,9,5,3,2,1,2,0,4356,18,0,3,1,2
2,66,4,2,10,5,8,4,2,2,0,4356,40,0,3,1,1
3,54,1,9,4,3,7,4,1,2,0,3900,40,0,3,1,0
4,41,1,2,10,4,1,3,1,2,0,3900,40,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,1,2,10,2,13,2,1,1,0,0,40,0,1,1,1
32557,27,1,7,12,1,12,5,1,2,0,0,38,0,2,1,2
32558,40,1,1,9,1,7,1,1,1,0,0,40,0,2,1,2
32559,58,1,1,9,5,4,4,1,2,0,0,40,0,3,1,0


In [108]:
df_completo.dropna(axis=0, thresh=2)

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,age_group,high_hours_per_week,smokes_or_drinks
0,90.000000,4.000000,1.000000,9.000000,5.000000,8.000000,2.000000,1.000000,2.000000,0.000000,4356.000000,40.000000,0.000000,3.000000,1.000000,0.000000
1,82.000000,1.000000,1.000000,9.000000,5.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4356.000000,18.000000,0.000000,3.000000,1.000000,2.000000
2,66.000000,4.000000,2.000000,10.000000,5.000000,8.000000,4.000000,2.000000,2.000000,0.000000,4356.000000,40.000000,0.000000,3.000000,1.000000,1.000000
3,54.000000,1.000000,9.000000,4.000000,3.000000,7.000000,4.000000,1.000000,2.000000,0.000000,3900.000000,40.000000,0.000000,3.000000,1.000000,0.000000
4,41.000000,1.000000,2.000000,10.000000,4.000000,1.000000,3.000000,1.000000,2.000000,0.000000,3900.000000,40.000000,0.000000,2.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65117,50.568933,-0.052854,8.035448,13.654599,3.578389,5.593822,2.780362,0.676604,0.451740,10860.622295,0.000000,45.054385,0.186932,2.010259,1.584521,0.608116
65118,45.531529,1.614166,8.092197,8.848909,3.325021,6.232248,2.352179,0.907096,2.384889,8455.303891,16.142255,66.552299,-0.272471,1.017383,0.728502,0.781720
65119,32.225932,3.026867,6.853070,12.917914,0.832072,7.037366,1.931776,0.617615,1.481953,1011.926449,0.000000,33.144402,0.289831,1.644973,0.928038,1.008781
65120,16.706573,1.686692,3.555098,13.967409,1.535759,2.284449,6.076681,1.104373,0.764239,447.945320,262.613545,37.895071,0.101088,1.448795,1.049217,0.015105


## Se asina el 80% de los datos para entrenar el modelo y el 20% para testear

In [99]:
# La entrada es de 16 elemento contando con x0
input_layer_size  = 16
#3 clases
num_labels = 3

X = df.iloc[:52097, :-1]
X_test = df.iloc[6512:, :-1]
y = df.iloc[:26048, df.columns.get_loc("native.country")]
y_test = df.iloc[6512:, df.columns.get_loc("native.country")]

#Cargando los valores ya procesados a las variables correspondientes

In [109]:
print(X.iloc[0,: ])
print(y)

age                      90
workclass                 4
education                 1
education.num             9
marital.status            5
occupation                8
relationship              2
race                      1
sex                       2
capital.gain              0
capital.loss           4356
hours.per.week           40
native.country            0
age_group                 3
high_hours_per_week       1
Name: 0, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
26043    0
26044    0
26045    0
26046    0
26047    0
Name: native.country, Length: 26048, dtype: int64
